In [41]:
import pandas as pd

In [42]:
man_df = pd.read_csv('data/caesar_fr.csv', encoding='utf-8', sep=';')
man_df=man_df.sample(frac=1)
caesar_male = man_df[man_df['sex'] == 1]

In [43]:
caesar_male=caesar_male.loc[:, (caesar_male != 0).any()]

In [44]:
import pandas as pd
import numpy as np

def remplacer_nan_par_moyenne(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            moyenne = df[col].mean(skipna=True)
            df[col] = df[col].fillna(moyenne)
    return df

caesar_malecaesar_malecaesar_male = remplacer_nan_par_moyenne(caesar_male)
caesar_male.head()

,taille,age,weight,sex,tour_de_cheville,longueur_du_bras,largeur_du_dos,tour_de_poitrine,tour_de_poitrine_mesure_a_la_main,hauteur_de_poitrine,...,hauteur_des_hanches,hauteur_des_genoux,tour_du_cou,largeur_d_epaule,hauteur_des_epaules,largeur_des_epaules_a_l_horizontales,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille
3170,190.0936,39.0,101.151016,1.0,27.4066,66.0908,44.5516,110.6932,NaN,140.7922,...,98.3996,54.6989,48.5902,44.5516,157.6070,53.2892,88.1888,65.6082,101.6000,110.3884
1722,169.5958,35.0,58.898921,1.0,25.0952,57.6072,38.3794,88.1888,NaN,120.5738,...,82.1944,46.4693,44.8056,38.3794,137.8204,42.3926,78.9940,50.5968,75.2094,103.4034
513,167.9956,48.0,74.842680,1.0,26.1874,61.3918,39.9034,94.9960,NaN,124.0028,...,82.3976,48.9966,48.4886,39.9034,137.2997,47.1932,82.1944,58.4962,86.5124,98.3996
1833,191.2874,19.0,89.498238,1.0,29.6926,64.2874,42.4434,100.7110,NaN,137.5664,...,94.3102,53.6194,51.2064,42.4434,155.5496,49.1998,87.5030,61.1886,94.2086,112.9030
2907,167.8940,55.0,85.728888,1.0,26.6954,62.5094,41.8084,111.7092,NaN,119.6086,...,82.7024,45.3390,46.5074,41.8084,136.8044,49.6062,84.4042,61.3918,95.8088,95.3008


In [45]:
caesar_male["largeur_des_hanches_ratio"] = caesar_male["largeur_des_hanches"] / caesar_male["taille"]
caesar_male["tour_de_taille_ratio"] = caesar_male["tour_de_taille"] / caesar_male["taille"]
caesar_male["largeur_d_epaule_ratio"] = caesar_male["largeur_d_epaule"] / caesar_male["taille"]

In [46]:
def classifier_morphologie(row):
    # Extraction des mesures
    #tour_ventre = row['tour_de_ventre_maximal']
    tour_taille = row['tour_de_taille_ratio']
    forme_poitrine = row['largeur_d_epaule_ratio']
    tour_cuisse = row['tour_de_cuisse']
    largeur_hanches = row['largeur_des_hanches_ratio']
    
    # --- Catégorie VENTRE (basée sur le ratio ventre/taille) ---
    if tour_taille < 0.462864:
        ventre = "plat"
    elif 0.462864 <= tour_taille<0.536128:
        ventre = "moyen"
    else:
        ventre = "rond"
    
    # --- Catégorie TORSE (basée sur le tour de poitrine seul ou comparé à la taille) ---
    if forme_poitrine < 0.224841:
        torse = "fin"
    elif 0.224841 <=forme_poitrine < 0.242234 :
        torse = "moyen"
    else:
        torse = "large"
    
    # --- Catégorie CUISSES (ratio cuisse / hanches) ---
    if largeur_hanches < 0.257434:
        cuisses = "fines"
    elif  0.257434 <=largeur_hanches<0.283177 :
        cuisses = "moyennes"
    else:
        cuisses = "larges"
    
    return pd.Series([ventre, torse, cuisses], 
                     index=['categorie_ventre', 'categorie_torse', 'categorie_cuisses'])
    
# Remplace `df` par le nom de ton DataFrame
caesar_male[['categorie_ventre', 'categorie_torse', 'categorie_cuisses']] = caesar_male.apply(classifier_morphologie, axis=1)

In [53]:
from sklearn.preprocessing import RobustScaler

# Colonnes à exclure
cols_to_exclude = ["taille","age","weight","categorie_ventre", "categorie_torse", "categorie_cuisses"]

# Colonnes à scaler (tout sauf celles à exclure)

caesar_male = caesar_male.dropna(how='all', axis=1)
cols_to_scale = [col for col in caesar_male.columns if col not in cols_to_exclude]
# On copie le DataFrame pour garder l'original intact
df_scaled = caesar_male.copy()

# On applique RobustScaler uniquement sur les colonnes numériques à scaler
scaler = RobustScaler()
df_scaled[cols_to_scale] = scaler.fit_transform(df_scaled[cols_to_scale])

# Résultat : df_scaled contient les colonnes scalées (sauf celles exclues)


In [55]:
df_scaled.head()

,taille,age,weight,sex,tour_de_cheville,longueur_du_bras,largeur_du_dos,tour_de_poitrine,hauteur_de_poitrine,hauteur_d_entrejambe,...,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille,largeur_des_hanches_ratio,tour_de_taille_ratio,largeur_d_epaule_ratio,categorie_ventre,categorie_torse,categorie_cuisses
3170,190.0936,39.0,101.151016,0.0,0.405063,0.453039,0.981264,0.777351,1.392701,0.775510,...,0.509259,0.985507,0.977444,0.820847,0.422987,0.504845,0.035561,moyen,moyen,moyennes
1722,169.5958,35.0,58.898921,0.0,-0.746835,-1.392265,-0.994346,-0.923225,-0.931387,-0.013605,...,-1.166667,-1.155797,-0.975564,-0.074919,-0.754818,-0.739731,-0.432005,plat,moyen,fines
513,167.9956,48.0,74.842680,0.0,-0.202532,-0.569061,-0.506541,-0.408829,-0.537226,-0.911565,...,-0.583333,-0.028986,-0.139098,-0.716612,0.445788,0.238110,0.218695,moyen,moyen,moyennes
1833,191.2874,19.0,89.498238,0.0,1.544304,0.060773,0.306467,0.023033,1.021898,0.775510,...,0.384259,0.355072,0.430451,1.143322,-0.473987,-0.069176,-0.687965,moyen,fin,fines
2907,167.8940,55.0,85.728888,0.0,0.050633,-0.325967,0.103215,0.854127,-1.042336,-0.962585,...,-0.180556,0.384058,0.548872,-1.114007,1.009782,0.999566,0.884636,rond,large,larges


In [67]:
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib


X_train=caesar_male[['taille', 'age', 'weight',
              'categorie_ventre', 'categorie_torse', 'categorie_cuisses']]

# Colonnes numériques et catégorielles
cols_num = ["taille","age","weight"]
cols_cat = ["categorie_ventre", "categorie_torse", "categorie_cuisses"]

# Pipeline pour les colonnes numériques : imputation par médiane + scaling
pipeline_num = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Pipeline pour les colonnes catégorielles : imputation par la valeur la plus fréquente + one-hot
pipeline_cat = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline globale de prétraitement
preprocessor = ColumnTransformer(transformers=[
    ("num", pipeline_num, cols_num),
    ("cat", pipeline_cat, cols_cat)
])

# Fit sur X_train (tu dois définir X_train comme DataFrame brut)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['taille', 'age', 'weight']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['categorie_ventre', 'categorie_torse',
                                  'categorie_cuisses'])])

In [68]:
print("Colonnes dans X_train :", X_train.columns.tolist())
print("Colonnes numériques :", cols_num)
print("Colonnes catégorielles :", cols_cat)

Colonnes dans X_train : ['taille', 'age', 'weight', 'categorie_ventre', 'categorie_torse', 'categorie_cuisses']
Colonnes numériques : ['taille', 'age', 'weight']
Colonnes catégorielles : ['categorie_ventre', 'categorie_torse', 'categorie_cuisses']


In [69]:
import joblib
import os

# 📁 Répertoire contenant les fichiers .pkl
dossier_modeles = "./modeles_tpot"  # modifie ce chemin si nécessaire

# 📌 Chargement de tous les modèles
modeles = {}
for nom_fichier in os.listdir(dossier_modeles):
    if nom_fichier.endswith(".pkl"):
        nom_cible = nom_fichier.replace("tpot_", "").replace(".pkl", "")
        chemin_complet = os.path.join(dossier_modeles, nom_fichier)
        modeles[nom_cible] = joblib.load(chemin_complet)

print(f"✅ {len(modeles)} modèles chargés :", list(modeles.keys()))

✅ 22 modèles chargés : ['hauteur_des_epaules', 'hauteur_des_genoux', 'hauteur_des_hanches', 'hauteur_de_la_taille', 'hauteur_de_poitrine', 'hauteur_d_entrejambe', 'largeur_des_epaules_a_l_horizontales', 'largeur_des_hanches', 'largeur_des_hanches_ratio', 'largeur_du_dos', 'largeur_d_epaule', 'largeur_d_epaule_ratio', 'longueur_avant', 'longueur_de_la_colonne_vertebrale_jusqu_au_poignet', 'longueur_du_bras', 'sex', 'tour_de_cheville', 'tour_de_cuisse', 'tour_de_poitrine', 'tour_de_taille', 'tour_de_taille_ratio', 'tour_du_cou']


In [70]:
import os

dossier_modeles = "./modeles_tpot"  # adapte ce chemin
noms_fichiers = [f for f in os.listdir(dossier_modeles) if f.endswith(".pkl")]

# Dossier de sauvegarde des pipelines complets
dossier_output = "pipelines_complets"
os.makedirs(dossier_output, exist_ok=True)

for nom_fichier in noms_fichiers:
    chemin_modele = os.path.join(dossier_modeles, nom_fichier)
    
    # Charger le modèle TPOT
    tpot_model = joblib.load(chemin_modele)
    
    # Créer le pipeline complet
    full_pipeline = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", tpot_model)
    ])
    
    # Sauvegarder le pipeline complet
    nom_fichier_pipeline = f"pipeline_{nom_fichier}"
    chemin_output = os.path.join(dossier_output, nom_fichier_pipeline)
    joblib.dump(full_pipeline, chemin_output)

    print(f"✔ Pipeline sauvegardé : {chemin_output}")

✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_des_epaules.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_des_genoux.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_des_hanches.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_de_la_taille.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_de_poitrine.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_d_entrejambe.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_des_epaules_a_l_horizontales.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_des_hanches.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_des_hanches_ratio.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_du_dos.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_d_epaule.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_d_epaule_ratio.pkl
✔ Pipelin

In [71]:
import os
import joblib
import pandas as pd

# 📁 1. Dossier où se trouvent les 22 pipelines
dossier_pipelines = "pipelines_complets"  # adapte selon ton cas

# 🧠 2. Charger tous les pipelines .pkl
pipelines = {}
for filename in os.listdir(dossier_pipelines):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        nom_variable = filename.replace("pipeline_tpot_", "").replace(".pkl", "")
        pipelines[nom_variable] = joblib.load(path)

# 📦 3. Exemple de nouvelles données
new_data = pd.DataFrame([{
    "taille": 187,
    "weight": 80,
    "age": 33,
    "categorie_ventre": "moyen",
    "categorie_cuisses": "medium",
    "categorie_torse": "large"
}])

# 🧮 4. Appliquer chaque pipeline à la même ligne de données
predictions = {}
for nom_variable, pipeline in pipelines.items():
    y_pred = pipeline.predict(new_data)
    predictions[nom_variable] = y_pred[0]  # extraire la valeur unique

# 🖨️ 5. Résultat
print("Prédictions :")
for var, val in predictions.items():
    print(f"{var} = {val:.2f}")


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\

Prédictions :
hauteur_des_epaules = 0.71
hauteur_des_genoux = 0.69
hauteur_des_hanches = 0.78
hauteur_de_la_taille = 0.91
hauteur_de_poitrine = 0.83
hauteur_d_entrejambe = 0.83
largeur_des_epaules_a_l_horizontales = 0.34
largeur_des_hanches = 0.34
largeur_des_hanches_ratio = -0.18
largeur_du_dos = 0.64
largeur_d_epaule = 0.64
largeur_d_epaule_ratio = -0.05
longueur_avant = 0.69
longueur_de_la_colonne_vertebrale_jusqu_au_poignet = 0.81
longueur_du_bras = 67.25
sex = 0.00
tour_de_cheville = 0.22
tour_de_cuisse = -0.16
tour_de_poitrine = -0.15
tour_de_taille = -0.19
tour_de_taille_ratio = -0.95
tour_du_cou = -0.11


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.ven

In [72]:
import os
import joblib
import pandas as pd

# 📁 Dossiers
dossier_pipelines = "pipelines_complets"
output_pred_path = "predictions_scaled.csv"

# 🧠 Chargement des pipelines
pipelines = {}
for filename in os.listdir(dossier_pipelines):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        nom_variable = filename.replace("pipeline_tpot_", "").replace(".pkl", "")
        pipelines[nom_variable] = joblib.load(path)

# 📦 Exemple de données
new_data = pd.DataFrame([{
    "taille": 187,
    "weight": 80,
    "age": 33,
    "categorie_ventre": "moyen",
    "categorie_cuisses": "medium",
    "categorie_torse": "large"
}])

# 🧮 Prédictions
predictions = {}
for nom_variable, pipeline in pipelines.items():
    y_pred_scaled = pipeline.predict(new_data)[0]
    predictions[nom_variable] = y_pred_scaled

# 💾 Sauvegarde dans un CSV
df_preds = pd.DataFrame([predictions])
df_preds.to_csv(output_pred_path, index=False)
print(f"✅ Prédictions scalées enregistrées dans : {output_pred_path}")


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\

✅ Prédictions scalées enregistrées dans : predictions_scaled.csv


In [73]:
import pandas as pd
import joblib
from sklearn.preprocessing import RobustScaler
import os

# 📁 Chargement du DataFrame d'entraînement (avec les vraies valeurs)
chemin_scalers = "scalers_y"
os.makedirs(chemin_scalers, exist_ok=True)

# 📄 Chargement des prédictions scalées
preds_scaled = pd.read_csv("predictions_scaled.csv")

# 🔁 Boucle sur chaque colonne prédite
preds_descaled = {}

for col in preds_scaled.columns:
    if col == "longueur_du_bras":
        preds_descaled[col] = preds_scaled[col][0]  # Pas scaled
        continue

    if col not in caesar_male.columns:
        print(f"⚠️ Colonne {col} absente du dataset original. Ignorée.")
        continue

    # Fit et sauvegarde du scaler
    scaler = RobustScaler()
    scaler.fit(caesar_male[[col]])
    joblib.dump(scaler, f"{chemin_scalers}/scaler_{col}.pkl")

    # Déscaler la prédiction
    scaled_val = preds_scaled[col][0]
    descaled_val = scaler.inverse_transform([[scaled_val]])[0][0]
    preds_descaled[col] = descaled_val

# ✅ Affichage des résultats déscalés
print("\nPrédictions déscalées (en cm) :")
for k, v in preds_descaled.items():
    print(f"{k} = {v:.2f}")



Prédictions déscalées (en cm) :
hauteur_des_epaules = 152.59
hauteur_des_genoux = 52.28
hauteur_des_hanches = 93.90
hauteur_de_la_taille = 111.05
hauteur_de_poitrine = 135.90
hauteur_d_entrejambe = 87.12
largeur_des_epaules_a_l_horizontales = 49.44
largeur_des_hanches = 49.44
largeur_des_hanches_ratio = 0.26
largeur_du_dos = 43.48
largeur_d_epaule = 43.48
largeur_d_epaule_ratio = 0.23
longueur_avant = 27.78
longueur_de_la_colonne_vertebrale_jusqu_au_poignet = 89.81
longueur_du_bras = 67.25
sex = 1.00
tour_de_cheville = 27.03
tour_de_cuisse = 57.57
tour_de_poitrine = 98.46
tour_de_taille = 85.81
tour_de_taille_ratio = 0.43
tour_du_cou = 46.77
